##Git and GitHub
Define access credentials and repo details.

The following environment variables (or secret keys) are assumed to be configured:
* GITHUB_USERNAME - github username
* GITHUB_USER_EMAIL - email configured for the github user
* GITHUB_TOKEN - github personal access token with relevant privileges (`repo` is enough)


In [1]:
from google.colab import userdata

github_user_email = userdata.get('GITHUB_USER_EMAIL')

In [3]:
token = userdata.get('GITHUB_TOKEN')
username = userdata.get('GITHUB_USERNAME')
repo = "mini-food-pic-classifier"

In [17]:
!git config --global user.name {username}
!git config --global user.email {github_user_email}
# !git config --global user.password ""

Get the latest code from the project repo.

Change the repo name and path as you need.

Create new repo in case you want to use a new repo.

In [4]:
!git clone https://{token}@github.com/{username}/{repo}

Cloning into 'mini-food-pic-classifier'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (4/4), done.


Change the working path to be inside the repo folder from this point and on.

In [5]:
%cd {repo}

/content/mini-food-pic-classifier


In [6]:
# project_name = "mini-food-pic-classifier"
project_path = "/content/" + repo

Currently assume the repo does not have the most updated version of this notebook, so copy it from the google drive:
* Connect to the drive
* Copy the latest notebook from the drive to its place in the local git repo

In [7]:
import os
load_notebooks_path = os.path.join(project_path, "notebooks")
if not os.path.exists(load_notebooks_path):
  os.mkdir(load_notebooks_path)

In [8]:
!cp "/content/drive/MyDrive/Colab Notebooks/youtube/Learn PyTorch for deep learning in a day/05_pytorch_going_modular/pytorch_going_modular_cell_mode.ipynb" ./notebooks/

In [9]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	notebooks/

nothing added to commit but untracked files present (use "git add" to track)


In [10]:
!git checkout -b cell_mode_get_data

Switched to a new branch 'cell_mode_get_data'


In [19]:
!git add notebooks
!git commit -m "Get data in cell mode"

[cell_mode_get_data 06fef79] Get data in cell mode
 1 file changed, 1 insertion(+)
 create mode 100644 notebooks/pytorch_going_modular_cell_mode.ipynb


In [20]:
!git status

On branch cell_mode_get_data
nothing to commit, working tree clean


In [22]:
!git push origin main

Everything up-to-date


##Cleaning the notebook from trials

In [ ]:
!git checkout -b remove_github_access_trials
!cp "/content/drive/MyDrive/Colab Notebooks/youtube/Learn PyTorch for deep learning in a day/05_pytorch_going_modular/pytorch_going_modular_cell_mode.ipynb" ./notebooks/
!git add notebooks/pytorch_going_modular_cell_mode.ipynb
!git commit -m "Remove github access trials"
!git push origin main

##0. Running a notebook in cell mode

This notebook is intended to run normally, one cell at a time.

## 1. Get data
We're going to start by downloading the `pizza_steak_sushi` datast with images of pizza, steak and sushi.

In [ ]:
import os
import zipfile

from pathlib import Path

import requests

# Setup path to data folder
data_path = Path(os.join(project_path, "data/"))
image_path = data_path / "pizza_steak_sushi"

# If the image folder doesn't exist, download it and prepare it...
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)

# Download pizza, steak, sushi data
with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Downloading pizza, steak, sushi data...")
    f.write(request.content)

# Unzip pizza, steak, sushi data
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
    print("Unzipping pizza, steak, sushi data...")
    zip_ref.extractall(image_path)

# Remove zip file
os.remove(data_path / "pizza_steak_sushi.zip")

In [ ]:
# Setup train and testing paths
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir